In [1]:
import os
import csv
import pandas as pd
# Get the current working directory (where your .ipynb file is saved)
folder_path = os.getcwd()

# Define the delimiter used in the .txt files (can be adjusted if needed)
actual_delimiter = ','  # Adjust if needed

# Loop through all files in the current folder
for filename in os.listdir(folder_path):
    # Check if the file has a .txt extension
    if filename.endswith('.txt'):
        # Construct the full path to the .txt file
        txt_file_path = os.path.join(folder_path, filename)
        
        # Create the corresponding .csv file path
        csv_file_path = os.path.join(folder_path, filename.replace('.txt', '.csv'))

        # Open both the input .txt file and the output .csv file
        with open(txt_file_path, 'r', encoding='utf-8') as txt_file, open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
            # Create a CSV reader to parse lines using the specified delimiter
            reader = csv.reader(txt_file, delimiter=actual_delimiter)
            
            # Create a CSV writer to write rows into the .csv file
            writer = csv.writer(csv_file)
            
            # Copy each row from the .txt file into the .csv file
            for row in reader:
                writer.writerow(row)

# Notify that the conversion is complete
print(" Conversion complete. Files saved in the same folder as this notebook.")


 Conversion complete. Files saved in the same folder as this notebook.


In [34]:

# Load datasets
demo = pd.read_csv('df_final_demo.csv')
experiment = pd.read_csv('df_final_experiment_clients.csv')
web1 = pd.read_csv('df_final_web_data_pt_1.csv')
web2 = pd.read_csv('df_final_web_data_pt_2.csv')

#Standardize formatting:
datasets = [demo, experiment, web1, web2]
for df in datasets:
    df.columns = df.columns.str.lower().str.strip()

# Merge web1 and web2
web = pd.concat([web1, web2], ignore_index=True)
web.drop_duplicates(inplace=True)

web.to_csv("web.csv", index=False)
print(" Merging complete and saved as web.csv")



 Merging complete and saved as web.csv


In [35]:
print(f' \n demo: \n {demo.head()}')
print(f' \n experiment: \n {experiment.head()}')
print(f' \n web: \n {web.head()}')
print(f' \n demo shape: \n {demo.shape}')
print(f' \n experiment shape: \n {experiment.shape}')
print(f' \n web shape: \n {web.shape}')

 
 demo: 
    client_id  clnt_tenure_yr  clnt_tenure_mnth  clnt_age gendr  num_accts  \
0     836976             6.0              73.0      60.5     U        2.0   
1    2304905             7.0              94.0      58.0     U        2.0   
2    1439522             5.0              64.0      32.0     U        2.0   
3    1562045            16.0             198.0      49.0     M        2.0   
4    5126305            12.0             145.0      33.0     F        2.0   

         bal  calls_6_mnth  logons_6_mnth  
0   45105.30           6.0            9.0  
1  110860.30           6.0            9.0  
2   52467.79           6.0            9.0  
3   67454.65           3.0            6.0  
4  103671.75           0.0            3.0  
 
 experiment: 
    client_id variation
0    9988021      Test
1    8320017      Test
2    4033851   Control
3    1982004      Test
4    9294070   Control
 
 web: 
    client_id            visitor_id                      visit_id process_step  \
0    9988021  58

In [36]:
#Standardize column names

def standardize_columns(df):
    df.columns = (
        df.columns
        .str.strip()                        # Remove leading/trailing spaces
        .str.lower()                        # Convert to lowercase
        .str.replace(' ', '_', regex=False) # Replace spaces with underscores
        .str.replace(r'\W', '', regex=True) # Remove special characters
    )
    return df


# Apply to all three DataFrames
demo = standardize_columns(demo)
experiment = standardize_columns(experiment)
web = standardize_columns(web)


# Define the rename mappings
demo = demo.rename(columns=        {'clnt_tenure_yr': 'tenure_year',
                                   'clnt_tenure_mnth': 'tenure_month', 
                                   'clnt_age': 'age',
                                   'gendr': 'gender', 
                                   'num_accts': 'num_accounts', 
                                   'bal':'balance', 
                                   'calls_6_mnth':'calls_6_months',
                                   'logons_6_mnth':'logons_6_months'})



In [37]:
# Quick structure check
for name, df in zip(['demo', 'experiment', 'web'], [demo, experiment, web]):
    print(f'\n{name.upper()} INFO:')
    print(df.info())
    print(df.describe(include="all"))



DEMO INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   client_id        70609 non-null  int64  
 1   tenure_year      70595 non-null  float64
 2   tenure_month     70595 non-null  float64
 3   age              70594 non-null  float64
 4   gender           70595 non-null  object 
 5   num_accounts     70595 non-null  float64
 6   balance          70595 non-null  float64
 7   calls_6_months   70595 non-null  float64
 8   logons_6_months  70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB
None
           client_id   tenure_year  tenure_month           age gender  \
count   7.060900e+04  70595.000000  70595.000000  70594.000000  70595   
unique           NaN           NaN           NaN           NaN      4   
top              NaN           NaN           NaN           NaN      U   
freq           

In [38]:
#Check unique values in each DataFrame

# List of the dataframes
dataframes = {'demo': demo, 'experiment': experiment, 'web': web}
# Loop through each DataFrame and its columns
for name, df in dataframes.items():
    print(f"\n--- Unique values in {name} ---")
    for col in df.columns:
        print(f"\n{col} - Unique values:")
        print(df[col].unique())


--- Unique values in demo ---

client_id - Unique values:
[ 836976 2304905 1439522 ...  333913 1573142 5602139]

tenure_year - Unique values:
[ 6.  7.  5. 16. 12. 30. 15.  8.  4.  3. 10. 11. 13. 14. 19. 18. 25. 23.
 21. 17.  9. 27. 24. 20. 28. 22. 29. 46. 26. 32. 33. 35.  2. 38. 31. 48.
 34. 36. 51. nan 47. 49. 37. 39. 40. 55. 42. 44. 43. 50. 54. 41. 45. 52.
 62.]

tenure_month - Unique values:
[ 73.  94.  64. 198. 145.  71.  66. 361. 369. 189. 100.  99.  67. 193.
  58.  47. 122. 136. 101. 103.  70. 152. 160.  80.  88.  86.  65.  68.
  93.  85. 168. 142. 228. 223. 305. 286. 285. 280. 254. 253. 278. 221.
 216. 212. 204. 260. 116. 130. 177. 159. 190.  89.  91. 129. 329.  92.
  82. 169. 239.  69. 178.  95.  63. 187. 288. 261. 251. 250.  56.  60.
 123. 175. 188.  59. 336. 149. 156. 330. 155. 167.  96. 242. 163.  81.
 182. 134. 113. 172. 120. 105. 271. 244. 200. 157.  78. 151. 121. 184.
 335. 357. 111. 185.  76. 148.  97.  83. 347. 324. 162. 194.  74. 186.
 300. 289. 295. 282. 270. 246. 22

In [39]:
# Null values check
print("demo null counts:\n", demo.isna().sum(), "\n")
print("experiment null counts:\n", experiment.isna().sum(), "\n")
print("web null counts:\n", web.isna().sum(), "\n")

demo null counts:
 client_id           0
tenure_year        14
tenure_month       14
age                15
gender             14
num_accounts       14
balance            14
calls_6_months     14
logons_6_months    14
dtype: int64 

experiment null counts:
 client_id        0
variation    20109
dtype: int64 

web null counts:
 client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64 



In [40]:
#Convert columns to appropriate dtypes

cols_to_convert = ['num_accounts','calls_6_months', 'logons_6_months']  # replace with your actual column names

demo[cols_to_convert] = demo[cols_to_convert].astype('Int64')  # allows NaNs


In [42]:
# Duplicates check
print(demo.duplicated().sum())
print(web.duplicated().sum())
print(experiment.duplicated().sum())

0
0
0


In [43]:
print("demo:")
print(demo.dtypes)
print("\nexperiment:")
print(experiment.dtypes)
print("\nweb:")
print(web.dtypes)

demo:
client_id            int64
tenure_year        float64
tenure_month       float64
age                float64
gender              object
num_accounts         Int64
balance            float64
calls_6_months       Int64
logons_6_months      Int64
dtype: object

experiment:
client_id     int64
variation    object
dtype: object

web:
client_id        int64
visitor_id      object
visit_id        object
process_step    object
date_time       object
dtype: object


In [44]:
demo['gender']

0        U
1        U
2        U
3        M
4        F
        ..
70604    U
70605    M
70606    F
70607    M
70608    F
Name: gender, Length: 70609, dtype: object

In [45]:

demo['gender'] = demo['gender'].astype(str).str.strip().str.upper()
gender_map = {
    'M': 'male',
    'F': 'female',
    'U': 'unknown',
    'X': 'unknown',  # Treat 'X' the same as 'U'
    'NAN': 'unknown',
}
demo['gender'] = demo['gender'].map(gender_map)

In [46]:
demo['gender'].unique()

array(['unknown', 'male', 'female'], dtype=object)

In [47]:
demo

,client_id,tenure_year,tenure_month,age,gender,num_accounts,balance,calls_6_months,logons_6_months
0,836976,6.0,73.0,60.5,unknown,2,45105.30,6,9
1,2304905,7.0,94.0,58.0,unknown,2,110860.30,6,9
2,1439522,5.0,64.0,32.0,unknown,2,52467.79,6,9
3,1562045,16.0,198.0,49.0,male,2,67454.65,3,6
4,5126305,12.0,145.0,33.0,female,2,103671.75,0,3
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,unknown,3,1411062.68,5,5
70605,8981690,12.0,148.0,31.0,male,2,101867.07,6,6
70606,333913,16.0,198.0,61.5,female,2,40745.00,3,3
70607,1573142,21.0,255.0,68.0,male,3,475114.69,4,4


In [48]:
# Remove rows where TenureYears > 42
demo = demo[demo['tenure_year'] <= 42]

demo

,client_id,tenure_year,tenure_month,age,gender,num_accounts,balance,calls_6_months,logons_6_months
0,836976,6.0,73.0,60.5,unknown,2,45105.30,6,9
1,2304905,7.0,94.0,58.0,unknown,2,110860.30,6,9
2,1439522,5.0,64.0,32.0,unknown,2,52467.79,6,9
3,1562045,16.0,198.0,49.0,male,2,67454.65,3,6
4,5126305,12.0,145.0,33.0,female,2,103671.75,0,3
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,unknown,3,1411062.68,5,5
70605,8981690,12.0,148.0,31.0,male,2,101867.07,6,6
70606,333913,16.0,198.0,61.5,female,2,40745.00,3,3
70607,1573142,21.0,255.0,68.0,male,3,475114.69,4,4


In [ ]:
experiment['variation'] = experiment['variation'].str.lower()
experiment['variation'].unique()

array(['test', 'control', nan], dtype=object)

In [50]:
experiment['variation'].value_counts(dropna=False)


variation
test       26968
control    23532
NaN        20109
Name: count, dtype: int64

In [51]:
experiment['variation'] = experiment['variation'].fillna('unknown')
experiment['variation'].unique()

array(['test', 'control', 'unknown'], dtype=object)

In [52]:
web['date_time'] = pd.to_datetime(web['date_time'])

In [53]:
web[['visitor_id', 'visit_id', 'process_step']] = web[['visitor_id', 'visit_id', 'process_step']].apply(lambda col: col.str.strip().str.lower())

In [54]:
# Original DataFrames stored in a dictionary
dfs = {
    "demo": demo,
    "dexperiment": experiment,
    "web": web
}

# Drop nulls from each DataFrame
for name in dfs:
    dfs[name] = dfs[name].dropna()
    print(f"\nNull values in {name}:")
    print(dfs[name].isnull().sum())


Null values in demo:
client_id          0
tenure_year        0
tenure_month       0
age                0
gender             0
num_accounts       0
balance            0
calls_6_months     0
logons_6_months    0
dtype: int64

Null values in dexperiment:
client_id    0
variation    0
dtype: int64

Null values in web:
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64


In [55]:
# Merge demo + experiment
base = pd.merge(demo, experiment, on='client_id', how='left') # Chose left to keep all clients from demo (even those not in the experiment).


# Merge with web Data
merged = pd.merge(web, base, on='client_id', how='inner') # This links every web event to client demographics and the A/B test group.



print(merged.shape)
print(merged.columns)


(443365, 14)
Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time',
       'tenure_year', 'tenure_month', 'age', 'gender', 'num_accounts',
       'balance', 'calls_6_months', 'logons_6_months', 'variation'],
      dtype='object')


After merging, the number of rows in merged should match the number of rows in your web logs that have matching client_ids in both demo and experiment.
expected_rows = number of rows in web1 + web2 
                that have matching client_ids in demo & experiment

In [58]:
#check null values 
null_counts = base.isnull().sum()
print("\nNull Value Counts:")
print(null_counts)


Null Value Counts:
client_id          0
tenure_year        0
tenure_month       0
age                0
gender             0
num_accounts       0
balance            0
calls_6_months     0
logons_6_months    0
variation          0
dtype: int64


In [60]:
#drop null values
base = base.dropna(subset=['age'])

In [61]:
#check null values 
null_counts = base.isnull().sum()
print("\nNull Value Counts:")
print(null_counts)


Null Value Counts:
client_id          0
tenure_year        0
tenure_month       0
age                0
gender             0
num_accounts       0
balance            0
calls_6_months     0
logons_6_months    0
variation          0
dtype: int64


In [62]:
#check null values 
null_counts = merged.isnull().sum()
print("\nNull Value Counts:")
print(null_counts)


Null Value Counts:
client_id           0
visitor_id          0
visit_id            0
process_step        0
date_time           0
tenure_year         0
tenure_month        0
age                12
gender              0
num_accounts        0
balance             0
calls_6_months      0
logons_6_months     0
variation           0
dtype: int64


In [63]:
#drop null values
merged = merged.dropna(subset=['age'])

In [64]:
#check null values 
null_counts = merged.isnull().sum()
print("\nNull Value Counts:")
print(null_counts)


Null Value Counts:
client_id          0
visitor_id         0
visit_id           0
process_step       0
date_time          0
tenure_year        0
tenure_month       0
age                0
gender             0
num_accounts       0
balance            0
calls_6_months     0
logons_6_months    0
variation          0
dtype: int64


In [65]:
merged.duplicated().sum()

np.int64(0)

In [66]:
demo.to_csv("demo.csv", index=False)
print(" Merging complete and saved as demo.csv")
base.to_csv("base.csv", index=False)
print(" Merging complete and saved as base.csv")
merged.to_csv('vanguard_merged.csv', index=False, encoding='utf-8')
print(" Merging complete and saved as vanguard_merged.csv")


 Merging complete and saved as demo.csv
 Merging complete and saved as base.csv
 Merging complete and saved as vanguard_merged.csv
